In this notebook, we see that `qubits` can help us with `encryption` (or other cryptographic tasks) by letting us securely distribute secret keys. There are classical methods for sharing random keys (e.g., RSA), but they have different guarantees about the security of the sharing.

We will be working with one of the most common `QKD protocols—BB84` but there are many others that we won’t have time to get into.

In [ ]:
# qkd.py: exchanging classical bits via qubits

#from interface import QuantumDevice, Qubit
#from simulator import SingleQubitSimulator

def prepare_classical_message(bit: bool, q: Qubit) -> None:
    if bit:
        q.x()
    """To prepare our qubit with the classical bit we want to send, we need as input the bit value and a qubit to use"""

def eve_measure(q: Qubit) -> bool:
    return q.measure()

def send_classical_bit(device: QuantumDevice, bit: bool) -> None:
    with device.using_qubit() as q:
        prepare_classical_message(bit, q)
        result = eve_measure(q)
        q.reset()
    assert result == bit

def qrng(device: QuantumDevice) -> bool:
    with device.using_qubit() as q:
        q.h()
        return q.measure()

Now add the functionality we need to our simulator to run listing

In [3]:
from abc import ABCMeta, abstractmethod
from contextlib import contextmanager

class Qubit(metaclass=ABCMeta):
    @abstractmethod
    def h(self): pass

    @abstractmethod
    def x(self): pass
    """We can model implementing the quantum NOT operation after the h operation."""

    @abstractmethod
    def measure(self) -> bool: pass

    @abstractmethod
    def reset(self): pass

class QuantumDevice(metaclass=ABCMeta):
    @abstractmethod
    def allocate_qubit(self) -> Qubit:
        pass

    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit):
        pass

    @contextmanager
    def using_qubit(self):
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)

Now that our interface for a qubit knows that we want an implementation of the x operation, let’s add that implementation!

In [ ]:
# simulator.py: adding x to the qubit simulator
KET_0 = np.array([
    [1],
    [0]
], dtype=complex)

H = np.array([
    [1, 1],
    [1, -1]
], dtype=complex) / np.sqrt(2)

X = np.array([
    [0, 1],
    [1, 0]
], dtype=complex)

class SimulatedQubit(Qubit):
    def __init__(self):
        self.reset()

    def h(self):
        self.state = H @ self.state

    def x(self):
        self.state = X @ self.state

    def measure(self) -> bool:
        pr0 = np.abs(self.state[0, 0]) ** 2
        sample = np.random.random() <= pr0
        return bool(0 if sample else 1)

    def reset(self):
        self.state = KET_0.copy()

class SingleQubitSimulator(QuantumDevice):
    available_qubits = [SimulatedQubit()]

    def allocate_qubit(self) -> SimulatedQubit:
        if self.available_qubits:
            return self.available_qubits.pop()

    def deallocate_qubit(self, qubit: SimulatedQubit):
        self.available_qubits.append(qubit)

Using our upgraded Python qubit simulator to share a secret classical bit with a qubit.

Using a fresh qubit, prepare it based on the classical bit value we want to send Eve. Eve then measures the qubit, and we can see if we both have the same classical bit value.

In [ ]:
# Sending classical bits with a single-qubit simulator

# We need a simulated qubit to use for our QRNG.
qrng_simulator = SingleQubitSimulator()

# Reusing the qrng function that we wrote in chapter 2, we can generate a random classical bit to use for our key.
key_bit = int(qrng(qrng_simulator))
 
# We’ll are using a new qubit simulator instance here for the key exchange, but strictly speaking, we don’t need to. We’ll see in chapter 4 how to expand the simulator to work with multiple qubits.
qkd_simulator = SingleQubitSimulator()

             with qkd_simulator.using_qubit() as q:
             prepare_classical_message(key_bit, q) 
             # We encode our classical bit in the qubit provided by qkd_simulator. If the classical bit was a 0, we do nothing to
             # qkd_simulator; and if the classical bit was a 1, we use the x method to change the qubit to the |1 〉 state.                         
             print(f"You prepared the classical key bit: {key_bit}")

             # Eve measures the qubit from qkd_simulator and then stores the bit value as eve_measurement
             eve_measurement = int(eve_measure(q)) 
             print(f"Eve measured the classical key bit: {eve_measurement}")

Is this secure? If you suspect it is not secure, you are definitely onto something. In the next section, we’ll discuss the security of our prototype secret-sharing scheme and look at ways to improve it.

We and Eve now have a way to send classical bits using qubits, but what happens if an adversary gets hold of that qubit? They could use the measure instruction to get the same classical data that Eve does. That’s a huge problem and would reasonably make us wonder why anyone would use qubits to share keys in the first place. Fortunately, quantum mechanics offers a way to make this exchange more secure! What are some modifications we could make to our protocol? For instance, we could represent a classical “0” message with a qubit in the |+ 〉 state and a “1” message with a qubit in the |− 〉 state.

In [6]:
def prepare_classical_message_plusminus(bit: bool, q: Qubit) -> None:
    if bit:
        q.x()
    q.h()

def eve_measure_plusminus(q: Qubit) -> bool:
    q.h()
    return q.measure()

def send_classical_bit_plusminus(device: QuantumDevice, bit: bool) -> None:
    with device.using_qubit() as q:
        prepare_classical_message_plusminus(bit, q)
        result = eve_measure_plusminus(q)
        assert result == bit

def send_classical_bit_wrong_basis(device: QuantumDevice, bit: bool) -> None:
    with device.using_qubit() as q:
        prepare_classical_message(bit, q)
        result = eve_measure_plusminus(q)
        assert result == bit, "Two parties do not have the same bit value"

Now we have two different ways of sending qubits that we and Eve could use when sending qubits. We call these two different ways of sending messages bases, and each contains two completely distinguishable (orthogonal) states.

Simulating the `BB84` protocol

Let's set up some functions that will simplify how we write out the full `BB84` protocol. We and Eve need to do things like sample random bits and prepare and measure the message qubit, separated here for clarity.

In [ ]:

# helper functions before the key exchange
from interface import QuantumDevice, Qubit
from simulator import SingleQubitSimulator
from typing import List

def sample_random_bit(device: QuantumDevice) -> bool:
    with device.using_qubit() as q:
        q.h()
        result = q.measure()
        q.reset() #Here we reset the qubit after measuring as we know we want to be able to use it more than once.
    return result

def prepare_message_qubit(message: bool, basis: bool, q: Qubit) -> None: # ❷ The qubit is encoded with the key bit value in the randomly selected basis. 
    if message:
        q.x()
    if basis:
        q.h()

def measure_message_qubit(basis: bool, q: Qubit) -> bool:
    if basis:
        q.h()
    result = q.measure()
    q.reset()  # ❸ Similar to sample_random_bit after Eve measures the message qubit, she should reset it because in the simulator, we will reuse it.
    return result

def convert_to_hex(bits: List[bool]) -> str: # ❹ To help condense the display of long binary keys, a helper function converts the representation to a shorter hex string. 
    return hex(int(
        "".join(["1" if bit else "0" for bit in bits]),
        2
    ))

In [8]:
# BB84 protocol for sending a classical bit

def send_single_bit_with_bb84(
    your_device: QuantumDevice,
    eve_device: QuantumDevice
    ) -> tuple:

    [your_message, your_basis] = [
        sample_random_bit(your_device) for _ in range(2)
    ]

    eve_basis = sample_random_bit(eve_device)

    with your_device.using_qubit() as q:
        prepare_message_qubit(your_message, your_basis, q)

        # QUBIT SENDING...

        eve_result = measure_message_qubit(eve_basis, q)

    return ((your_message, your_basis), (eve_result, eve_basis))

In [9]:
# BB84 protocol for exchanging a key with Eve.

def simulate_bb84(n_bits: int) -> list:
    your_device = SingleQubitSimulator()
    eve_device = SingleQubitSimulator()

    key = []
    n_rounds = 0

    while len(key) < n_bits:
        n_rounds += 1
        ((your_message, your_basis), (eve_result, eve_basis)) = \
            send_single_bit_with_bb84(your_device, eve_device)

        if your_basis == eve_basis:
            assert your_message == eve_result
            key.append(your_message)

    print(f"Took {n_rounds} rounds to generate a {n_bits}-bit key.")

    return key

The key is now in the bag, so we can move on to using the key and the one-time pad encryption algorithm to send a secret message!

In [ ]:

# Using BB84 and one-time pad encryption

def apply_one_time_pad(message: List[bool], key: List[bool]) -> List[bool]:
    return [
        message_bit ^ key_bit # The ^ operator is a bitwise XOR in Python. This applies a single bit of our key as a one-time pad to our message text.
        for (message_bit, key_bit) in zip(message, key)
    ]

if __name__ == "__main__":
    print("Generating a 96-bit key by simulating BB84...")
    key = simulate_bb84(96)
    print(f"Got key
                           {convert_to_hex(key)}.")

    message = [
        1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 0, 1, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 0, 1, 1
    ]

    print(f"Using key to send secret message: {convert_to_hex(message)}.")

    encrypted_message = apply_one_time_pad(message, key)
    print(f"Encrypted message:
                {convert_to_hex(encrypted_message)}.")

    decrypted_message = apply_one_time_pad(encrypted_message, key)
    print(f"Eve decrypted to get:
             {convert_to_hex(decrypted_message)}.")

❶ Since our basis and Eve’s basis will agree roughly half the time, it should take about two rounds of BB84 for each bit of key we want to generate. 

❷ The exact key we generate will be different every time we run the BB84 simulation—that’s a huge part of the point of the protocol, after all! 

❸ The message we get by writing down each of the Unicode code points for “ ” 

❹ When we combine our secret message with the key we got earlier, using the key as a one-time pad, our message is scrambled.
 
❺ When Eve uses the same key, she gets back our original secret message.

In [1]:
# qkd.py: Defines functions needed to perform a basic quantum key exchange
# protocol.
##
# Copyright (c) Sarah Kaiser and Chris Granade.
# Code sample from the book "Learn Quantum Computing with Python and Q#" by
# Sarah Kaiser and Chris Granade, published by Manning Publications Co.
# Book ISBN 9781617296130.
# Code licensed under the MIT License.
##